this is the code used to fine tune my model using the data send by the client

In [17]:
import os
import sys
import argparse
import shutil
from pathlib import Path
from ultralytics import YOLO
import yaml
from datetime import datetime

In [18]:
def setup_paths():
    """Setup and validate paths"""
    base_dir = Path.cwd()
    
    paths = {
        'base_dir': base_dir,
        'client_dataset': base_dir / 'client-dataset',
        'runs_dir': base_dir / 'runs',
        'existing_model': base_dir / 'person_detection' / 'game_characters' / 'weights' / 'epoch20.pt'
    }
    
    # Check if client dataset exists
    if not paths['client_dataset'].exists():
        raise FileNotFoundError(f"Client dataset not found at {paths['client_dataset']}")
    
    # Check if existing model exists
    if not paths['existing_model'].exists():
        print(f"Warning: Existing model not found at {paths['existing_model']}")
        print("Available models:")
        weights_dir = paths['existing_model'].parent
        if weights_dir.exists():
            for weight_file in weights_dir.glob('*.pt'):
                print(f"  - {weight_file.name}")
    
    return paths

In [19]:
def validate_dataset(dataset_path):
    """Validate dataset structure and return statistics"""
    data_yaml = dataset_path / 'data.yaml'
    if not data_yaml.exists():
        raise FileNotFoundError(f"data.yaml not found in {dataset_path}")
    
    # Load dataset configuration
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    print(f"Dataset configuration:")
    print(f"  - Classes: {data_config.get('names', 'Not specified')}")
    print(f"  - Train path: {data_config.get('train', 'Not specified')}")
    print(f"  - Val path: {data_config.get('val', 'Not specified')}")
    print(f"  - Test path: {data_config.get('test', 'Not specified')}")
    
    # Count files in each split
    splits = ['train', 'val', 'test']
    for split in splits:
        if split in data_config:
            split_path = dataset_path / split
            if split_path.exists():
                images_path = split_path / 'images'
                labels_path = split_path / 'labels'
                
                if images_path.exists() and labels_path.exists():
                    image_count = len(list(images_path.glob('*')))
                    label_count = len(list(labels_path.glob('*')))
                    print(f"  - {split.capitalize()}: {image_count} images, {label_count} labels")
                else:
                    print(f"  - {split.capitalize()}: Missing images or labels directory")
    
    return data_config

In [20]:
def create_training_config(data_config, output_dir):
    """Create a training-specific data.yaml file with absolute paths"""
    training_config = data_config.copy()
    
    # Convert relative paths to absolute paths
    base_path = Path.cwd() / 'client-dataset'
    
    for split in ['train', 'val', 'test']:
        if split in training_config:
            # Make paths absolute
            training_config[split] = str(base_path / split / 'images')
    
    # Save training config
    training_yaml = output_dir / 'training_data.yaml'
    with open(training_yaml, 'w') as f:
        yaml.dump(training_config, f, default_flow_style=False)
    
    print(f"Created training configuration at {training_yaml}")
    return training_yaml

In [21]:
def setup_training_directory():
    """Create a timestamped training directory"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"client_finetune_{timestamp}"
    
    runs_dir = Path.cwd() / 'runs' / 'detect'
    output_dir = runs_dir / run_name
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Training output directory: {output_dir}")
    return output_dir

In [22]:
def train_model(model_path, data_yaml, output_dir, epochs=50, batch_size=16, patience=10):
    """Train the YOLO model"""
    print(f"\nStarting training...")
    print(f"Model: {model_path}")
    print(f"Data: {data_yaml}")
    print(f"Output: {output_dir}")
    print(f"Epochs: {epochs}")
    print(f"Batch size: {batch_size}")
    print(f"Patience: {patience}")
    print("-" * 50)
    
    # Load the model
    model = YOLO(str(model_path))
    
    # Train the model
    results = model.train(
        data=str(data_yaml),
        epochs=epochs,
        batch=batch_size,
        patience=patience,
        save=True,
        save_period=5,  # Save checkpoint every 5 epochs
        project=str(output_dir.parent.parent),  # runs/
        name=output_dir.name,
        exist_ok=True,
        # Hyperparameters for fine-tuning
        lr0=0.001,      # Lower learning rate for fine-tuning
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        # Augmentation (moderate for fine-tuning)
        degrees=10.0,   # Reduced rotation
        translate=0.1,  # Reduced translation
        scale=0.5,      # Reduced scaling
        shear=2.0,      # Reduced shear
        flipud=0.0,     # No vertical flip
        fliplr=0.5,     # 50% horizontal flip
        mosaic=0.5,     # Reduced mosaic probability
        mixup=0.1,      # Light mixup
        # Optimizer
        optimizer='AdamW',
        close_mosaic=10,  # Disable mosaic in last 10 epochs
        amp=True,       # Automatic Mixed Precision
        # Validation
        val=True,
        plots=True,
        verbose=True
    )
    
    print(f"\nTraining completed!")
    print(f"Results saved to: {output_dir}")
    
    # Print key metrics
    if hasattr(results, 'results_dict'):
        metrics = results.results_dict
        print(f"\nFinal Metrics:")
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                print(f"  {key}: {value:.4f}")
    
    return results

In [23]:
def copy_best_weights(output_dir, destination=None):
    """Copy the best weights to a convenient location"""
    weights_dir = output_dir / 'weights'
    if weights_dir.exists():
        best_weights = weights_dir / 'best.pt'
        if best_weights.exists():
            if destination is None:
                destination = Path.cwd() / 'client_finetuned_best.pt'
            
            shutil.copy2(best_weights, destination)
            print(f"Best weights copied to: {destination}")
            return destination
        else:
            print("Warning: best.pt not found in weights directory")
    else:
        print("Warning: weights directory not found")
    
    return None

In [24]:
# Set arguments manually instead of using parser
epochs = 100
batch_size = 32
patience = 15
model = None 
output_name = None

try:
    print("=" * 60)
    print("YOLO CLIENT DATASET FINE-TUNING")
    print("=" * 60)
    
    # Setup paths
    print("\n1. Setting up paths...")
    paths = setup_paths()
    
    # Use custom model if provided
    if model:
        custom_model = Path(model)
        if custom_model.exists():
            paths['existing_model'] = custom_model
            print(f"Using custom model: {custom_model}")
        else:
            print(f"Warning: Custom model {custom_model} not found, using default")
    
    # Validate dataset
    print("\n2. Validating dataset...")
    data_config = validate_dataset(paths['client_dataset'])
    
    # Setup training directory
    print("\n3. Setting up training environment...")
    output_dir = setup_training_directory()
    
    # Create training configuration
    print("\n4. Creating training configuration...")
    training_yaml = create_training_config(data_config, output_dir)
    
    # Train the model
    print("\n5. Training model...")
    results = train_model(
        model_path=paths['existing_model'],
        data_yaml=training_yaml,
        output_dir=output_dir,
        epochs=epochs,
        batch_size=batch_size,
        patience=patience
    )
    
    # Copy best weights
    print("\n6. Finalizing...")
    final_output_name = output_name or 'client_finetuned_best.pt'
    final_model = copy_best_weights(output_dir, Path.cwd() / final_output_name)
    
    print("\n" + "=" * 60)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"Training directory: {output_dir}")
    if final_model:
        print(f"Final model: {final_model}")
    print(f"Use this model for inference in your notebooks or applications.")
    
except Exception as e:
    print(f"\nError during training: {e}")
    sys.exit(1)

YOLO CLIENT DATASET FINE-TUNING

1. Setting up paths...

2. Validating dataset...
Dataset configuration:
  - Classes: ['enemy']
  - Train path: ../train/images
  - Val path: ../valid/images
  - Test path: ../test/images
  - Train: 708 images, 708 labels
  - Val: 149 images, 149 labels
  - Test: 167 images, 167 labels

3. Setting up training environment...
Training output directory: /home/kronbii/repos/CV-aim-assist/runs/detect/client_finetune_20250917_210235

4. Creating training configuration...
Created training configuration at /home/kronbii/repos/CV-aim-assist/runs/detect/client_finetune_20250917_210235/training_data.yaml

5. Training model...

Starting training...
Model: /home/kronbii/repos/CV-aim-assist/person_detection/game_characters/weights/epoch20.pt
Data: /home/kronbii/repos/CV-aim-assist/runs/detect/client_finetune_20250917_210235/training_data.yaml
Output: /home/kronbii/repos/CV-aim-assist/runs/detect/client_finetune_20250917_210235
Epochs: 100
Batch size: 32
Patience: 15
-

train: Scanning /home/kronbii/repos/CV-aim-assist/client-dataset/train/labels... 708 images, 0 backgrounds, 0 corrupt: 100%|██████████| 708/708 [00:00<00:00, 2386.90it/s]


train: New cache created: /home/kronbii/repos/CV-aim-assist/client-dataset/train/labels.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 87.3±51.1 MB/s, size: 30.5 KB)


val: Scanning /home/kronbii/repos/CV-aim-assist/client-dataset/val/labels... 149 images, 1 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<00:00, 951.56it/s]

val: New cache created: /home/kronbii/repos/CV-aim-assist/client-dataset/val/labels.cache


...

     49/100       3.9G      1.606      1.166      1.172          5        640: 100%|██████████| 23/23 [00:04<00:00,  4.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  6.49it/s]

                   all        149        161      0.708      0.379      0.443      0.186
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 34, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



49 epochs completed in 0.079 hours.
Optimizer stripped from /home/kronbii/repos/CV-aim-assist/runs/client_finetune_20250917_210235/weights/last.pt, 6.2MB
Optimizer stripped from /home/kronbii/repos/CV-aim-assist/runs/client_finetune_20250917_210235/weights/best.pt, 6.2MB

Validating /home/kronbii/repos/CV-aim-assist/runs/client_finetune_20250917_210235/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7815MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.06it/s]


                   all        149        161       0.72      0.512      0.574      0.255
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /home/kronbii/repos/CV-aim-assist/runs/client_finetune_20250917_210235

Training completed!
Results saved to: /home/kronbii/repos/CV-aim-assist/runs/detect/client_finetune_20250917_210235

Final Metrics:
  metrics/precision(B): 0.7205
  metrics/recall(B): 0.5124
  metrics/mAP50(B): 0.5741
  metrics/mAP50-95(B): 0.2546
  fitness: 0.2866

6. Finalizing...

TRAINING COMPLETED SUCCESSFULLY!
Training directory: /home/kronbii/repos/CV-aim-assist/runs/detect/client_finetune_20250917_210235
Use this model for inference in your notebooks or applications.
